In [ ]:
import sire as sr

In [ ]:
import sys

sys.path.insert(1, "/home/anna/Documents/code/python")

# import libraries

from scipy.stats import sem as sem
import sys
import glob
import itertools as it

import logging

logging.getLogger().setLevel(logging.INFO)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *



In [ ]:
# ligands = [lig.split("/")[-1].split(".")[0] for lig in glob.glob("/home/anna/Documents/placement_test/syk/inputs/ligands/neutral/*")]
# perturbations = [f"{a}~{b}" for a,b in it.product(ligands, ligands)]
# graph = network_graph(ligands, perturbations, ligands_folder="/home/anna/Documents/placement_test/syk/inputs/ligands/neutral")
# graph.draw_all_ligands(figsize=(10,30))

In [ ]:
perts, ligs = pipeline.utils.get_info_network(
    "/backup/42_reruns/tyk2/execution_model/network_combined_reruns_ejm42.dat"
)

for eng in all_analysis_object.engines:
    other_results = glob.glob(
        f"/backup/42_reruns/tyk2/outputs_extracted/results/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(other_results, name=f"{eng}_reruns")
    # all_analysis_object.plot_other_results(
    #     name=f"{eng}_reruns", engine=eng, pert_val="pert"
    # )
    all_analysis_object.plot_bar_ddG(
        engine=[eng, f"{eng}_reruns"], **{"values": perts}
    )
    res1 = all_analysis_object._stats_object.compute_mue("pert", y=eng)
    res2 = all_analysis_object._stats_object.compute_mue("pert", y=f"{eng}_reruns")
    res3 = all_analysis_object._stats_object.compute_mue(
        "pert", y=eng, x=f"{eng}_reruns"
    )
    print(
        f"og vs exp is {res1[0]}, reruns vs exp is {res2[0]}, and compared to each other theya re {res3[0]}"
    )

all_analysis_object.plot_bar_ddG(
    engine=all_analysis_object.engines + all_analysis_object.other_results_names,
    **{"values": perts},
)

In [ ]:
bench_folder = f"/home/anna/Documents/placement_test"
protein = "cmet"
main_dir = f"{bench_folder}/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{main_dir}/inputs/{protein}.yml"
output_folder = f"{main_dir}/outputs_extracted"

In [ ]:
prot_file = f"{main_dir}/execution_model/protocol_int2.dat"
pipeline_prot = pipeline_protocol(prot_file, auto_validate=True)
pipeline_prot.name()

In [ ]:
all_analysis_object = analysis_network(
    output_folder,
    exp_file=exp_file,
    # net_file=net_file,
    analysis_prot=ana_file,
    method = pipeline_prot.name(), # if the protocol had a name
    engines="SOMD"
)

In [ ]:
all_analysis_object.change_name("SOMD", "delta1_coulomb1_dielectric82.0")


In [ ]:
all_analysis_object.calc_pert_dict

In [ ]:
other_results_files = glob.glob(
    f"{output_folder}/results/freenrg_*_SOMD_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)

all_analysis_object.compute_other_results(other_results_files, method="flarenet", name="delta1_coulomb1_dielectric78.3")

In [ ]:
all_analysis_object.exper_pert_dict

In [ ]:
all_analysis_object.plot_bar_ddG(
    engines=["delta1_coulomb1_dielectric78.3","delta1_coulomb1_dielectric82.0"],
    **{"values":["lig_18~lig_4","lig_4~lig_18"]}
    )

In [ ]:
all_analysis_object.perturbing_atoms_and_overlap(
    prep_dir=f"/home/anna/Documents/benchmark/{protein}_benchmark/prep",
    outputs_dir=f"{main_dir}/outputs_extracted",
)

In [ ]:
all_analysis_object._initialise_stats_object()
mae = all_analysis_object._stats_object.compute_mue(y="GROMACS", pert_val="val")
stats_string = f"MAE: {mae[0]:.2f} +/- {mae[1]:.2f} kcal/mol"
print(stats_string)

In [ ]:
calculate only the experimental results for some of the perturbations
protein = "mcl1"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
# net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

output_folder = validate.folder_path(f"{main_dir}/diff_methods_analysis", create=True)
results_folder = f"{main_dir}/diff_methods_analysis/results"

all_analysis_object = analysis_network(exp_file=exp_file)

all_analysis_object.perturbations = [
    "lig_27~lig_59",
    "lig_27~lig_48",
    "lig_28~lig_47",
    "lig_35~lig_52",
    "lig_38~lig_48",
    "lig_43~lig_45",
    "lig_53~lig_56",
    "lig_30~lig_40",
    "lig_27~lig_48",
]
all_analysis_object.ligands = get_ligands_from_perts(all_analysis_object.perturbations)
all_analysis_object.get_experimental()

In [ ]:
# cycle closures
all_analysis_object.compute_cycle_closures()
cycles = all_analysis_object.network_graph.cycle_closures()

plotting with stats as title

In [ ]:
# MAE calculate
for eng in all_analysis_object.engines:
    mae = all_analysis_object._stats_object.compute_mue(y=eng, pert_val="pert")
    stats_string = f"MAE: {mae[0]:.2f} +/- {mae[1]:.2f} kcal/mol"
    all_analysis_object.plot_scatter_ddG(
        engine=eng,
        **{
            "title": stats_string,
            "key": False,
            #    "save": f"/home/anna/Desktop/{eng}"
        },
    )